# LOESS

## Enfoque General

**1. Preprocesamiento:**

Se extraen las columnas de provisión (ya transformadas a porcentaje multiplicándolas por 100) y se filtran aquellos meses en los que se disponga de datos (por ejemplo, donde el valor es mayor a cero).

**2. Ajuste de la Curva Suave (LOESS):**

Para cada cliente se usa la función lowess del paquete statsmodels para ajustar una curva suave a los datos. Esto permite capturar relaciones complejas sin asumir una forma funcional predefinida.

**3. Estimación de la Tendencia:**

Una vez que se obtiene la curva suavizada, se calcula la derivada aproximada (usando, por ejemplo, la función np.gradient) con respecto al tiempo. La derivada en cada punto indica la tasa de cambio local de la provisión. Se puede resumir la tendencia tomando la derivada promedio (o evaluando la derivada en puntos clave, como el medio del intervalo) para decidir si, en promedio, la provisión aumenta o disminuye a lo largo del tiempo.

**4. Interpretación:**

* **Derivada promedio positiva:** Indica que, en promedio, el porcentaje de provisión está aumentando con el tiempo.
* **Derivada promedio negativa:** Indica que, en promedio, está disminuyendo.
* **Derivada promedio cercana a cero:** Sugiere que no hay cambios importantes en la tendencia, o que la variación es muy irregular.

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

def calcular_tendencia_no_lineal(df):
    """
    Calcula la tendencia no lineal en el porcentaje de provisión (provision/saldo * 100)
    para cada cliente usando un ajuste LOESS y estima la derivada promedio de la curva suavizada.
    
    Se añaden las siguientes columnas al DataFrame:
      - tendencia_no_lineal: Derivada promedio de la curva suavizada (análogo a la pendiente en un modelo no lineal).
      - meses_con_datos: Número de meses con datos válidos (provisión > 0).
    
    Parámetros
    ----------
    df : pd.DataFrame
        DataFrame que contiene 'cliente_id' y columnas 'provision_mes1' a 'provision_mes12'.
        Se asume que cada valor en provision_mesX ya es el cociente provision/saldo (sin multiplicar por 100).
    
    Retorna
    -------
    df : pd.DataFrame
        El DataFrame original con las nuevas columnas añadidas.
    """
    
    # Lista de columnas de provisión
    meses_cols = [f'provision_mes{i}' for i in range(1, 13)]
    
    # Inicialización de las nuevas columnas
    df["tendencia_no_lineal"] = np.nan
    df["meses_con_datos"] = np.nan
    
    # Iterar sobre cada cliente
    for idx, row in df.iterrows():
        # Extraer y convertir a float los valores, multiplicándolos por 100 para expresarlos en porcentaje.
        # Esto transforma el cociente (provision/saldo) a un porcentaje.
        provisiones = np.array([float(row[col]) * 100 for col in meses_cols])
        
        # Identificar los índices (meses) en que la provisión es mayor a cero.
        idx_validos = np.where(provisiones > 0)[0]
        # Renumerar los meses válidos para que empiecen en 1
        if len(idx_validos) > 0:
            meses_validos = (idx_validos + 1).astype(float)
        else:
            meses_validos = np.array([])
        
        # Filtrar los valores de provisión correspondientes
        provisiones_validas = provisiones[idx_validos]
        
        # Guardar el número de meses con datos
        df.at[idx, "meses_con_datos"] = len(meses_validos)
        
        if len(meses_validos) > 1:
            # Ajuste LOESS para suavizar la relación no lineal
            # El parámetro frac determina la fracción de datos usados en cada estimación local
            frac = 0.6  # Este parámetro se puede ajustar según la variabilidad de los datos
            loess_result = sm.nonparametric.lowess(provisiones_validas, meses_validos, frac=frac, return_sorted=True)
            x_suave = loess_result[:, 0]
            y_suave = loess_result[:, 1]
            
            # Calcular la derivada aproximada de la curva suavizada
            derivada = np.gradient(y_suave, x_suave)
            
            # Se toma como tendencia la derivada promedio
            tendencia_promedio = np.mean(derivada)
        else:
            tendencia_promedio = np.nan
        
        df.at[idx, "tendencia_no_lineal"] = tendencia_promedio
    
    return df

# Ejemplo de uso:
# Cargar el DataFrame (se asume que ya contiene cliente_id y provision_mes1 ... provision_mes12)
df = pd.read_csv("datos_provision.csv")

# Calcular la tendencia no lineal y actualizar el DataFrame
df_actualizado = calcular_tendencia_no_lineal(df)

# Guardar o visualizar el DataFrame actualizado
df_actualizado.to_csv("datos_provision_actualizados_no_lineal.csv", index=False)
print(df_actualizado.head())


## Explicación y Consideraciones

**1. Transformación a Porcentaje:**

Se multiplica cada valor extraído de las columnas por 100. Esto convierte la razón provisión/saldo a un porcentaje, facilitando su interpretación.

**2. Filtrado de Datos Válidos:**

Solo se consideran los meses en los que el valor de provisión es mayor a cero, ya que meses en cero pueden representar la ausencia de información útil.


**3. Uso de LOESS:**

La función lowess de statsmodels ajusta una curva suave a los datos.
El parámetro frac controla la cantidad de puntos utilizados en cada estimación local y puede ajustarse según la densidad y variabilidad de los datos.

**4. Estimación de la Derivada:**

Se utiliza np.gradient para aproximar la derivada de la curva suavizada. La derivada en cada punto indica la tasa de cambio local del porcentaje de provisión. Se promedia esta derivada para obtener una medida global de la tendencia.

**5. Interpretación de la Tendencia No Lineal:**

* Tendencia positiva (derivada promedio > 0): El porcentaje de provisión, en promedio, está aumentando a lo largo del tiempo.

* Tendencia negativa (derivada promedio < 0): Está disminuyendo.

* Tendencia cercana a cero: No hay cambios significativos o la variación es muy irregular.


Si los datos muestran una tendencia no lineal, el uso de técnicas de suavizado como LOESS es una alternativa robusta a la regresión lineal. Esta metodología permite capturar patrones complejos y estimar una “pendiente” o tasa de cambio promedio a partir de la derivada de la curva suavizada, ofreciendo así una visión más precisa de la evolución del porcentaje de provisión en relación con el saldo del cliente.